In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


###Necessary Libraries

In [2]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [3]:
df_control = pd.read_csv('/content/gdrive/MyDrive/Carephasia_Data_Cleaned/Cleaned_All/control_data_clean.csv')
df_patient = pd.read_csv('/content/gdrive/MyDrive/Carephasia_Data_Cleaned/Cleaned_All/patient_data_clean.csv')
df_patient.head()

,text
0,well I could I could talk youknow
1,remember
2,when it was like the end of August
3,it was infact it was Labor Day
4,but I youknow I I when I was sick I didnt I I ...


In [4]:
df_patient.isnull().sum()
# df_control.isnull().sum()

text    445
dtype: int64

In [5]:
# Remove empty rows
df_patient.dropna(inplace=True)
df_control.dropna(inplace=True)


In [6]:
df_patient.reset_index(drop=True, inplace=True)
df_control.reset_index(drop=True,inplace= True)
df_patient

,text
0,well I could I could talk youknow
1,remember
2,when it was like the end of August
3,it was infact it was Labor Day
4,but I youknow I I when I was sick I didnt I I ...
...,...
38861,the the the the a the peanut butter and jelly
38862,jelly
38863,jelly on the step s
38864,and there you go


##**Preprocessing**

In [13]:
# baaje result ashchilo still a result
nlp = spacy.load('en_core_web_sm')
def calculate_avg_semantic_error(text):
    # Parse the text into a spaCy doc object
    doc = nlp(text)

    # Initialize a list to store similarity scores
    similarity_scores = []

    # Iterate over each token in the doc
    for token in doc:
        # Check if the token is a noun or verb
        if token.pos_ in ['NOUN', 'VERB']:
            # Get the head noun or verb of the phrase containing the token
            head = token.head

            # Calculate the similarity between the token and its head
            similarity = token.similarity(head)

            # If the similarity is valid, append it to the list
            if similarity is not None:
                similarity_scores.append(similarity)

    # Calculate the average similarity score
    if similarity_scores:
        avg_similarity = sum(similarity_scores) / len(similarity_scores)
    else:
        avg_similarity = None

    return avg_similarity

In [35]:
import spacy
import pandas as pd

# Load the spaCy language model for English
nlp = spacy.load('en_core_web_sm')

# Define a function to detect semantic errors using spaCy's similarity scoring
def detect_semantic_errors(text):
    # Parse the text into a spaCy doc object
    doc = nlp(text)

    # Initialize an empty list to store potential semantic errors
    semantic_errors = []

    # Iterate over each token in the doc
    for token in doc:
        # Check if the token is a noun or verb
        if token.pos_ in ['NOUN', 'VERB']:
            # Get the head noun or verb of the phrase containing the token
            head = token.head

            # Calculate the similarity between the token and its head
            similarity = token.similarity(head)

            # If the similarity is low, consider it a potential semantic error
            if similarity < 0.7:
                semantic_errors.append((token.text, head.text, similarity))

    return semantic_errors

# Read the texts from aphasia patients from a file or data source


# Extract the text content from each row
texts = df_patient['text'].tolist()

# Process each text to detect semantic errors
semantic_errors_all = []
for text in texts:
    semantic_errors_text = detect_semantic_errors(text)
    semantic_errors_all.append([text, semantic_errors_text])

# Process and store the detected semantic errors
# ...


<ipython-input-35-2f4c8450bbbf>:23: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = token.similarity(head)


In [36]:
semantic_errors_all
# if nothing works, I will use this, and in places where there is no score, it will be 0.

[['well I could I could talk youknow', []],
 ['remember', []],
 ['when it was like the end of August',
  [('end', 'like', 0.08853886276483536)]],
 ['it was infact it was Labor Day', [('infact', 'was', -0.14694038033485413)]],
 ['but I youknow I I when I was sick I didnt I I I was in intensive care when I go off with the',
  [('youknow', 'did', 0.46505993604660034),
   ('care', 'in', 0.05908917635679245),
   ('go', 'was', 0.5047502517700195)]],
 ['like a', []],
 ['only four weeks on', [('weeks', 'on', 0.16842228174209595)]],
 ['but I I I I', []],
 ['well and and and my family called and xxx and xxx',
  [('family', 'called', 0.02210778184235096),
   ('called', 'well', -0.0012176684103906155)]],
 ['and then', []],
 ['they have a', []],
 ['they shipped me out in an ambulance ten oclock at night',
  [('ambulance', 'in', 0.008961651474237442),
   ('oclock', 'ambulance', 0.345887154340744),
   ('night', 'at', 0.00870057288557291)]],
 ['and I and I and I couldnt w I I was in a wheelchair youkn

In [7]:
!pip install https://huggingface.co/spacy/en_core_web_md/resolve/main/en_core_web_md-any-py3-none-any.whl

# Using spacy.load().
import spacy
nlp = spacy.load("en_core_web_md")

# Importing as module.
import en_core_web_md
nlp = en_core_web_md.load()
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load spaCy with pre-trained word embeddings
nlp = spacy.load("en_core_web_md")

# Example datasets
aphasia_data = ["The cat is on the mat.", "He walk dog.", "I eat an apple."]
control_data = ["The cat is sleeping.", "She walks her dog.", "I enjoy eating apples."]

def calculate_context_embedding(sentence):
    # Calculate the average word embedding for the entire sentence
    words = nlp(sentence)
    word_embeddings = [word.vector for word in words if word.has_vector]

    if not word_embeddings:
        # If no word has a vector, return a vector of zeros
        return np.zeros((1, 300))

    return np.mean(word_embeddings, axis=0).reshape(1, -1)

def calculate_intrasentence_cosine_similarity_with_context(data):
    similarities = []

    for sentence in data:
        # Get context embedding for the sentence
        embedding = calculate_context_embedding(sentence)

        # Calculate cosine similarity between the sentence and itself
        similarity = cosine_similarity(embedding, embedding)[0, 0]
        similarities.append(similarity)

    return similarities

# Calculate intra-sentence similarities with context for aphasia and control datasets
aphasia_similarities = calculate_intrasentence_cosine_similarity_with_context(aphasia_data)
control_similarities = calculate_intrasentence_cosine_similarity_with_context(control_data)

# Print the results
print("Intra-Sentence Similarities with Context for Aphasia Dataset:")
for i, similarity in enumerate(aphasia_similarities):
    print(f"Sentence {i + 1}: {aphasia_data[i]} - Similarity: {similarity}")

print("\nIntra-Sentence Similarities with Context for Control Dataset:")
for i, similarity in enumerate(control_similarities):
    print(f"Sentence {i + 1}: {control_data[i]} - Similarity: {similarity}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 11.8 MB/s eta 0:00:00
Intra-Sentence Similarities with Context for Aphasia Dataset:
Sentence 1: The cat is on the mat. - Similarity: 1.0
Sentence 2: He walk dog. - Similarity: 1.0000001192092896
Sentence 3: I eat an apple. - Similarity: 0.9999999403953552

Intra-Sentence Similarities with Context for Control Dataset:
Sentence 1: The cat is sleeping. - Similarity: 0.9999999403953552
Sentence 2: She walks her dog. - Similarity: 1.0000001192092896
Sentence 3: I enjoy eating apples. - Similarity: 1.0000001192092896


In [17]:
txt = ['a', 'b', 'c']
for i, text in enumerate(txt):
  print(i, text)

0 a
1 b
2 c


In [18]:
text_patient = df_patient['text'].tolist()
aphasia_similarities = calculate_intrasentence_cosine_similarity_with_context(text_patient)
# control_similarities = calculate_intrasentence_cosine_similarity_with_context(control_data)

# Print the results
# print("Intra-Sentence Similarities with Context for Aphasia Dataset:")
# for i, similarity in enumerate(aphasia_similarities):
#     print(f"{aphasia_data[i-1]} - Sim: {similarity}")

# print("\nIntra-Sentence Similarities with Context for Control Dataset:")
# for i, similarity in enumerate(control_similarities):
#     print(f"Sentence {i + 1}: {control_data[i]} - Similarity: {similarity}")
aphasia_similarities

[1.0,
 0.99999994,
 0.99999994,
 0.9999999,
 1.0000001,
 0.99999994,
 0.99999994,
 1.0,
 1.0000001,
 0.99999994,
 1.0000001,
 1.0,
 0.9999998,
 0.99999994,
 0.9999999,
 1.0,
 1.0,
 0.9999999,
 1.0,
 0.99999994,
 0.99999994,
 1.0,
 1.0000001,
 0.99999994,
 1.0,
 1.0000001,
 1.0000001,
 0.9999998,
 1.0,
 1.0000001,
 0.99999994,
 0.99999994,
 0.9999999,
 1.0,
 1.0000001,
 1.0,
 1.0000001,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0000001,
 1.0000001,
 1.0,
 0.99999994,
 1.0000001,
 1.0,
 1.0,
 1.0,
 1.0000001,
 0.9999999,
 1.0000001,
 0.99999994,
 1.0,
 0.99999994,
 1.0000002,
 1.0000001,
 0.9999999,
 0.9999998,
 0.99999994,
 1.0000001,
 0.99999976,
 1.0000001,
 0.99999994,
 1.0000001,
 1.0000001,
 0.99999994,
 1.0,
 1.0,
 0.99999994,
 0.99999994,
 0.9999998,
 1.0000001,
 0.99999994,
 1.0,
 1.0,
 1.0000001,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0000001,
 1.0000001,
 1.0,
 0.99999994,
 0.99999976,
 0.99999994,
 0.99999994,
 1.0000001,
 0.99999994,
 1.0,
 1.0,
 0.99999994,
 1.0000001,
 